[View in Colaboratory](https://colab.research.google.com/github/LauraMartinus/ukuxhumana/blob/master/tensor2tensor/notebooks/t2t_problem.ipynb)

# Welcome to the [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor) Dataset Colab!

Tensor2Tensor, or T2T for short, is a library of deep learning models and datasets designed to make deep learning more accessible and [accelerate ML research](https://research.googleblog.com/2017/06/accelerating-deep-learning-research.html).

**This colab shows you how to add your own dataset to T2T so that you can train one of the several preexisting models on your newly added dataset!**

For a tutorial that covers all the broader aspects of T2T using existing datasets and models, please see this [IPython notebook](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb).

In [0]:
#@title
# Copyright 2018 Google LLC.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

>[Welcome to the Tensor2Tensor Dataset Colab!](#scrollTo=Wd48fv-zDMe6)

>>[Installation & Setup](#scrollTo=Urn4QmNfI3hw)

>>[Define the Problem](#scrollTo=LUoP57gOjlk9)

>>>[Run t2t_datagen](#scrollTo=Q1xBmlrFLSPX)

>>[Viewing the generated data.](#scrollTo=MCqJhdnYgiG-)

>>>[tf.python_io.tf_record_iterator](#scrollTo=uNpohcPXKsLN)

>>>[Using tf.data.Dataset](#scrollTo=6o_1BHGQC5w5)

>>[Terminology](#scrollTo=xRtfC0sHBlSo)

>>>[Problem](#scrollTo=xRtfC0sHBlSo)

>>>[Modalities](#scrollTo=xRtfC0sHBlSo)



## Installation & Setup


We'll install T2T and TensorFlow.

We also need to setup the directories where T2T will:

*   Generate the dataset and write the TFRecords file representing the training and the eval set, vocabulary files etc `DATA_DIR`
*   Run the training, keep the graph and the checkpoint files `OUTPUT_DIR` and
*   Use as a scratch directory to download your dataset from a URL, unzip it, etc. `TMP_DIR`

In [0]:
#@title Run for installation.

! pip install -q -U tensor2tensor
! pip install -q tensorflow

In [0]:
#@title Run this only once - Sets up TF Eager execution.

import tensorflow as tf

# Enable Eager execution - useful for seeing the generated data.
tf.enable_eager_execution()

In [0]:
#@title Setting a random seed.

from tensor2tensor.utils import trainer_lib

# Set a seed so that we have deterministic outputs.
RANDOM_SEED = 301
trainer_lib.set_random_seed(RANDOM_SEED)

In [0]:
#@title Run for setting up directories.

import os

# Setup and create directories.
DATA_DIR = os.path.expanduser("/tmp/t2t/data")
OUTPUT_DIR = os.path.expanduser("/tmp/t2t/output")
TMP_DIR = os.path.expanduser("/tmp/t2t/tmp")

# Create them.
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(OUTPUT_DIR)
tf.gfile.MakeDirs(TMP_DIR)

## Define the `Problem`

To simplify our setting our input text sampled randomly from [a, z] - each sentence has between [3, 20] words with each word being [1, 8] characters in length.

Example input: "olrkpi z cldv xqcxisg cutzllf doteq" -- this will be generated by `sample_sentence()`

Our output will be the input words sorted according to length.

Example output: "z cldv doteq olrkpi xqcxisg cutzllf" -- this will be processed by `target_sentence()`

Let's dive right into our first problem -- we'll explain as we go on.

Take some time to read each line along with its comments -- or skip them and come back later to clarify your understanding.

In [0]:
#@title Define `sample_sentence()` and `target_sentence(input_sentence)`
import random
import string

def sample_sentence():
    # Our sentence has between 3 and 20 words
    num_words = random.randint(3, 20)
    words = []
    for i in range(num_words):
        # Our words have between 1 and 8 characters.
        num_chars = random.randint(1, 8)
        chars = []
        for j in range(num_chars):
            chars.append(random.choice(string.ascii_lowercase))
        words.append("".join(chars))
    return " ".join(words)

def target_sentence(input_sentence):
    words = input_sentence.split(" ")
    return " ".join(sorted(words, key=lambda x: len(x)))

In [0]:
# `Problem` is the base class for any dataset that we want to add to T2T -- it
# unifies the specification of the problem for generating training data,
# training, evaluation and inference.
#
# All its methods (except `generate_data`) have reasonable default
# implementations.
#
# A sub-class must implement `generate_data(data_dir, tmp_dir)` -- this method
# is called by t2t-trainer or t2t-datagen to actually generate TFRecord dataset
# files on disk.
from tensor2tensor.data_generators import problem

# Certain categories of problems are very common, like where either the input or
# output is text, for such problems we define an (abstract) sub-class of
# `Problem` called `Text2TextProblem` -- this implements `generate_data` in
# terms of another function `generate_samples`. Sub-classes must override
# `generate_samples` and `is_generate_per_split`.
from tensor2tensor.data_generators import text_problems

# Every non-abstract problem sub-class (as well as models and hyperparameter
# sets) must be registered with T2T so that T2T knows about it and can look it
# up when you specify your problem on the commandline to t2t-trainer or
# t2t-datagen.
#
# One uses:
# `register_problem` for a new Problem sub-class.
# `register_model` for a new T2TModel sub-class.
# `register_hparams` for a new hyperparameter set. All hyperparameter sets
# typically extend `common_hparams.basic_params1` (directly or indirectly).
from tensor2tensor.utils import registry

from tensor2tensor.data_generators import text_encoder

from tensor2tensor.data_generators import translate

# By default, when you register a problem (or model or hyperparameter set) the
# name with which it gets registered is the 'snake case' version -- so here
# the Problem class `SortWordsAccordingToLengthRandom` will be registered with
# the name `sort_words_according_to_length_random`.
#
# One can override this default by actually assigning a name as follows:
# `@registry.register_problem("my_awesome_problem")`

# End-of-sentence marker.
EOS = text_encoder.EOS_ID


_ENTN_TRAIN_DATASETS = [
    [
        "https://github.com/LauraMartinus/ukuxhumana/blob/master/data/eng_tswane/eng_tswane.train.tar.gz?raw=true",
        (
            "entn_parallel.train.en",
            "entn_parallel.train.tn"
        )
    ]
]

_ENTN_TEST_DATASETS = [
    [
        "https://github.com/LauraMartinus/ukuxhumana/blob/master/data/eng_tswane/eng_tswane.dev.tar.gz?raw=true",
        (
            "entn_parallel.dev.en",
            "entn_parallel.dev.tn"
        )
    ]
]


@registry.register_problem
class TranslateEntnRma(translate.TranslateProblem):
  """Problem spec for WMT English-Tswane translation."""

  @property
  def approx_vocab_size(self):
    return 2**15  # 32768

  @property
  def vocab_filename(self):
    return "vocab.entn.%d" % self.approx_vocab_size


  def source_data_files(self, dataset_split):
    train = dataset_split == problem.DatasetSplit.TRAIN
    return _ENTN_TRAIN_DATASETS if train else _ENTN_TEST_DATASETS




That's it!

To use this with `t2t-trainer` or `t2t-datagen`, save it to a directory, add an `__init__.py` that imports it, and then specify that directory with `--t2t_usr_dir`.

i.e. as follows:

```
$ t2t-datagen \
  --problem=sort_words_according_to_length_random \
  --data_dir=/tmp/t2t/data \
  --tmp_dir=/tmp/t2t/tmp \
  --t2t_usr_dir=/tmp/t2t/usr

```

However, we'll generate the data from the colab itself as well -- this is what `t2t-datagen` essentially does.

## Generate the data.

We will now generate the data by calling `Problem.generate_data()` and inspect it.

In [12]:
entn = TranslateEntnRma()

entn.generate_data(DATA_DIR, TMP_DIR)

INFO:tensorflow:Downloading https://github.com/LauraMartinus/ukuxhumana/blob/master/data/eng_tswane/eng_tswane.train.tar.gz?raw=true to /tmp/t2t/tmp/eng_tswane.train.tar.gz?raw=true
100% completed
INFO:tensorflow:Successfully downloaded eng_tswane.train.tar.gz?raw=true, 7524431 bytes.
INFO:tensorflow:Generating vocab file: /tmp/t2t/data/vocab.entn.32768
INFO:tensorflow:Generating vocab from: [['https://github.com/LauraMartinus/ukuxhumana/blob/master/data/eng_tswane/eng_tswane.train.tar.gz?raw=true', ('entn_parallel.train.en', 'entn_parallel.train.tn')]]
INFO:tensorflow:Not downloading, file already found: /tmp/t2t/tmp/eng_tswane.train.tar.gz?raw=true
INFO:tensorflow:Reading file: entn_parallel.train.en
INFO:tensorflow:Reading file: entn_parallel.train.tn
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 1174
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 633
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 685
INFO:tensorflo

## Viewing the generated data.

`tf.data.Dataset` is the recommended API for inputting data into a TensorFlow graph and the `Problem.dataset()` method returns a `tf.data.Dataset` object.


In [13]:
tfe = tf.contrib.eager

Modes = tf.estimator.ModeKeys

# We can iterate over our examples by making an iterator and calling next on it.
eager_iterator = tfe.Iterator(entn.dataset(Modes.EVAL, DATA_DIR))
example = eager_iterator.next()

input_tensor = example["inputs"]
target_tensor = example["targets"]

# The tensors are actually encoded using the generated vocabulary file -- you
# can inspect the actual vocab file in DATA_DIR.
print("Tensor Input: " + str(input_tensor))
print("Tensor Target: " + str(target_tensor))

INFO:tensorflow:Reading data files from /tmp/t2t/data/translate_entn_rma-dev*
INFO:tensorflow:partition: 0 num_data_files: 1
Tensor Input: tf.Tensor([9993    3   84   26 4506   13    4 5214 3118    2    1], shape=(11,), dtype=int64)
Tensor Target: tf.Tensor([  167   156     3     8 20275     7  2308    27  1339     2     1], shape=(11,), dtype=int64)


In [15]:
from tensor2tensor import problems
problems.available()

/usr/local/lib/python2.7/dist-packages/tensor2tensor/data_generators/video_generated.py:33: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "<decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/applica

['algorithmic_addition_binary40',
 'algorithmic_addition_decimal40',
 'algorithmic_cipher_shift200',
 'algorithmic_cipher_shift5',
 'algorithmic_cipher_vigenere200',
 'algorithmic_cipher_vigenere5',
 'algorithmic_identity_binary40',
 'algorithmic_identity_decimal40',
 'algorithmic_multiplication_binary40',
 'algorithmic_multiplication_decimal40',
 'algorithmic_reverse_binary40',
 'algorithmic_reverse_binary40_test',
 'algorithmic_reverse_decimal40',
 'algorithmic_reverse_nlplike32k',
 'algorithmic_reverse_nlplike8k',
 'algorithmic_shift_decimal40',
 'algorithmic_sort_problem',
 'audio_timit_characters_tune',
 'audio_timit_tokens8k_test',
 'audio_timit_tokens8k_tune',
 'babi_qa_concat_all_tasks_10k',
 'babi_qa_concat_all_tasks_1k',
 'babi_qa_concat_task10_10k',
 'babi_qa_concat_task10_1k',
 'babi_qa_concat_task11_10k',
 'babi_qa_concat_task11_1k',
 'babi_qa_concat_task12_10k',
 'babi_qa_concat_task12_1k',
 'babi_qa_concat_task13_10k',
 'babi_qa_concat_task13_1k',
 'babi_qa_concat_task14

In [18]:
from tensor2tensor.utils import registry
registry.list_hparams()

['imagetransformer_b10l_dr03_moe_tpu',
 'lstm_seq2seq',
 'img2img_transformer2d_n24',
 'imagetransformer2d_base_8l_8_16_big',
 'imagetransformer_sep_channels_8l_8h',
 'img2img_transformer2d_n31',
 'afx_small_bfloat16',
 'imagetransformer_tiny_tpu',
 'slicenet_1',
 'transformer_big',
 'universal_transformer_dwa_base',
 'img2img_transformer2d_tiny',
 'adaptive_universal_transformer_with_sru_base',
 'slicenet_1tiny',
 'transformer_teeny',
 'universal_transformer_big',
 'universal_transformer_mix_after_ut_base',
 'imagetransformer_b10l_4h_big_uncond_dr01_tpu',
 'vqa_recurrent_self_attention_mix_before_ut',
 'transformer_tpu_with_conv',
 'next_frame_emily',
 'vqa_attention_feature_batch1024_numglimps1',
 'autoencoder_ordered_discrete',
 'vqa_attention_feature_hidden1024',
 'imagetransformer_base_8l_8h_big_cond_dr03_dan',
 'mtf_image_transformer_tiny_8gpu',
 'adaptive_universal_transformer_concat_tiny',
 'adaptive_universal_transformer_base',
 'imagetransformer1d_base_8l_64by64',
 'imagetran

In [0]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment, create_hparams

# Setup and create directories.
DATA_DIR = os.path.expanduser("/tmp/t2t/data")
OUTPUT_DIR = os.path.expanduser("/tmp/t2t/output")
TMP_DIR = os.path.expanduser("/tmp/t2t/tmp")

hparams = create_hparams('transformer_base_single_gpu')
hparams.batch_size = 1024
hparams.learning_rate_warmup_steps = 45000
hparams.learning_rate = .4

RUN_CONFIG = create_run_config(
      model_dir=OUTPUT_DIR,
      keep_checkpoint_max=3
)

exp_fn = create_experiment(
    run_config = RUN_CONFIG,
    hparams=hparams,
    model_name='transformer',
    problem_name='translate_entn_rma',
    data_dir=DATA_DIR,
    train_steps=125000,
    eval_steps=100
)
exp_fn.train_and_evaluate()

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 3, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb909728690>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_protocol': None, '_save_checkpoints_steps': 1000, '_keep_checkpoint_every_n_hours': 10000, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
  }
}
, '_model_dir': '/tmp/t2t/output', 'use_tpu': False, '_tf_random_seed': None, '_mas

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/t2t/output/model.ckpt.
INFO:tensorflow:loss = 9.278147, step = 1


## To be continued ...

Stay tuned for additions to this notebook for adding problems with non-text modalities like Images, Audio and Video!